## Libraries

In [9]:
library(igraph) # for gra§phs

## Functions for evolution

In [10]:
seed <- function(nodes.state, n.seeds = 1, method = 'random'){
    
    N <- length(nodes.state)
    
    if (method == 'random'){
        seeds.id <- sample.int(N, n.seeds)
    }
    
    nodes.state[seeds.id] <- 1
    
    return(nodes.state)
}

In [11]:
#this function is meanted for networks that do not change their structure during the evolution of the dynamics
update <- function(nodes.state, nodes.threshold, nodes.neighbors){
    
    N <- length(nodes.state)
    
    nodes.state.in <- nodes.state
    
    nodes.state.out <- nodes.state
    
    for (i in 1:N){
    
        if (nodes.state[i] != 1){
            
            if(length(nodes.neighbors[[i]])!=0){
        
                if(sum(nodes.state.in[nodes.neighbors[[i]]])/length(nodes.neighbors[[i]]) >= nodes.threshold[i]){
                    nodes.state.out[i] <- 1
                }
            }          
        
        }
    
    }
    
    return(nodes.state.out) 
}


# SIMULATION

## Parameters of the simulations for FIRST PLOT

In [14]:
n <- 10000 #number of nodes
threshold.list <- seq(from = 0.1, to = 0.3, by=0.02)
z.list <- seq(from=1, to=15, by=1)
max.update <- 100   #maximum number of updates
n.seeds <- 1 # number of initial seeds
cascade.fraction.threshold <- 0.2 #fraction of nodes to be set to 1 to consider a cascade happening
n.run <- 10 #number of runs for each set of parameters
filename <- 'ERG_simulation_plot5_sigma0.05.csv'
sd <- 0.05

## Simulations for FIRST PLOT

In [15]:
write.table(list('nodes', 'max.update', 'n.seeds', 'threshold', 'z', 'time', 'n.run', 'cascade.fraction'),
                    file = filename,
                    append = TRUE,
                    sep = ',', 
                    row.names = FALSE,
                    col.names = FALSE)


for (phi in threshold.list){
    
    for (z in z.list){
               
        for(k in 1:n.run){
            
            cascade <- 0
            
            nodes.threshold <- rnorm(n=n, mean =phi , sd=sd)

            p=z/n

            g <- erdos.renyi.game(n = n,
                         p.or.m = p,  
                         type = "gnp",
                         directed = FALSE,
                         loops = FALSE)

            nodes.neighbors <- list()
            for (i in 1:n){
                nodes.neighbors[i] <- list(neighbors(g, i))
            }


            nodes.state <- numeric(n) #initializing to zero all nodes state
            t <- 0   #initial timeù

            nodes.state <- seed(nodes.state, n.seeds) #inserting some seeds
            t <- 1   #time at which happen the seed

            for (j in 1:max.update){

                nodes.state.previous <- nodes.state

                nodes.state <- update(nodes.state, nodes.threshold, nodes.neighbors)

                t <- t+1

                if(identical(nodes.state, nodes.state.previous)){
                    break
                }
            }

            cascade <- sum(nodes.state)/n
         

            write.table(list(n, max.update, n.seeds, phi, z, t, k, cascade),
            file = filename,
            append = TRUE,
            sep = ',', 
            row.names = FALSE,
            col.names = FALSE)
            
            if(cascade>0.1){
                break
            }
            
        }
    }
}